Official documentation of the Chacha20 algorithm: https://www.rfc-editor.org/rfc/rfc8439.html

# **This implementation of encryption is not secure, it is for study purposes only!**

In [38]:
def rotate_left(value: int, roll: int, bits = 32):
    """Perform a left bit rotation."""
    return ((value << roll) & (2**bits - 1)) | (value >> (bits - roll))

def rotate_right(value: int, roll: int, bits = 32):
    """Perform a right bit rotation."""
    return ((value >> roll) | (value << (bits - roll))) & (2**bits - 1)

# Example usage:
value = 0x7998bfda # 32-bit
roll_amount = 7  # Number of positions to rotate

# Perform left rotation
left_rotated_value = rotate_left(value, roll_amount)
print(f"Left rotated value: 0x{left_rotated_value:08x}")

Left rotated value: 0xcc5fed3c


In [39]:
test_list = [
    0x879531e0, 0xc5ecf37d, 0x516461b1, 0xc9a62f8a,
    0x44c20ef3, 0x3390af7f, 0xd9fc690b, 0x2a5f714c,
    0x53372767, 0xb00a5631, 0x974c541a, 0x359e9963,
    0x5c971061, 0x3d631689, 0x2098d9d6, 0x91dbd320
]

MASK_32BIT = 0xffffffff  # 32 bites mask

def quarterround(state, num1: int,num2: int,num3: int,num4: int):

    a = state[num1]
    b = state[num2]
    c = state[num3]
    d = state[num4]

    a += b
    a = a & MASK_32BIT
    d ^= a
    d = rotate_left(d,16)

    c += d
    c = c & MASK_32BIT
    b ^= c
    b = rotate_left(b,12)

    a += b
    a = a & MASK_32BIT
    d ^= a
    d = rotate_left(d,8)

    c += d
    c = c & MASK_32BIT
    b ^= c
    b = rotate_left(b,7)

    state[num1] = a
    state[num2] = b
    state[num3] = c
    state[num4] = d


# TEST:
quarterround(test_list, 2, 7, 8, 13)

def print_state(state):
    for i, value in enumerate(state):
        if ((i % 4 == 0) & (i != 0)):
            print()
        print(f'0x{value:08x}\t', end=' ')
        #print(f'{value}\t', end=' ')
    print()

print_state(test_list)


0x879531e0	 0xc5ecf37d	 0xbdb886dc	 0xc9a62f8a	 
0x44c20ef3	 0x3390af7f	 0xd9fc690b	 0xcfacafd2	 
0xe46bea80	 0xb00a5631	 0x974c541a	 0x359e9963	 
0x5c971061	 0xccc07c79	 0x2098d9d6	 0x91dbd320	 


In [ ]:
def my_chacha20_block(key, counter, nonce,test = False):

    constants = [
        0x61707865, 0x3320646e, 0x79622d32, 0x6b206574
    ]
    key = input_converter(key)
    nonce = input_converter(nonce)

    # Initialize state
    state = constants + key + [counter] + nonce
    initial_state = list(state)  # Create a copy of the state

    if test:
      print("add the original state")
      print_state(state)

    # Perform 10 rounds of inner block operation
    for _ in range(10):
        my_inner_block(state)

    # Add initial state to current state
    state = [(x + y) & MASK_32BIT for x, y in zip(state, initial_state)]

    if test:
      print("after block operations")
      print_state(state)

    serialized_state = serialize_state(state)

    if test:
      print("serialize")
      print_serialized_block(serialized_state)

    return serialized_state

def my_inner_block (state):
    quarterround(state, 0, 4, 8, 12)
    quarterround(state, 1, 5, 9, 13)
    quarterround(state, 2, 6, 10, 14)
    quarterround(state, 3, 7, 11, 15)
    quarterround(state, 0, 5, 10, 15)
    quarterround(state, 1, 6, 11, 12)
    quarterround(state, 2, 7, 8, 13)
    quarterround(state, 3, 4, 9, 14)


def serialize_state(state):
    serialized = bytearray()
    for value in state:
        serialized.extend(value.to_bytes(4, byteorder='little'))
    #print(serialized)
    return serialized

def deserialize_state(serialized):
    state = []
    for i in range(0, len(serialized), 4):
        value_bytes = serialized[i:i+4]
        value = int.from_bytes(value_bytes, byteorder='little')
        state.append(value)
    return state

def print_serialized_block(serialized):
    for i in range(0, len(serialized), 16):
        chunk = serialized[i:i+16]
        if(i != len(serialized) - (len(serialized) % 16)):
            print(f'{i:03} ', ' '.join(f'{byte:02x}' for byte in chunk), ' ', ''.join(chr(byte) if 32 <= byte <= 126 else '.' for byte in chunk))
        else:
            print(f'{i:03} ', ' '.join(f'{byte:02x}' for byte in chunk), ' '.join(f'  ' for _ in range( 16-len(chunk))) ,' ', ''.join(chr(byte) if 32 <= byte <= 126 else '.' for byte in chunk))

def input_converter(string):
    # Remove the spaces from the input string and divide it into 8-character parts
    string = string.replace(' ', '')

    hex_blocks8 = [string[i:i+2] for i in range(0, len(string), 2)]
    hex_blocks32 = [hex_blocks8[i:i+4] for i in range(0, len(hex_blocks8), 4)]

    # Within a given block, the inputs are in reverse order, so they must be reversed
    hex_blocks32_ordered = [block[::-1] for block in hex_blocks32]
    hex_blocks = [''.join(slice) for slice in hex_blocks32_ordered]

    # From each block we form ints
    output = [int(x, 16) for x in hex_blocks]

    return output

In [41]:
def my_chacha20_encrypt(key, counter, nonce, plaintext):
    encrypted_message = bytearray()
    plaintext_len = len(plaintext)

    # Process 64 byte blocks
    for j in range(plaintext_len // 64):
        key_stream = my_chacha20_block(key, counter + j, nonce)
        block = bytearray(plaintext[j*64:(j+1)*64], 'utf-8')
        encrypted_block = bytes([block[i] ^ key_stream[i] for i in range(64)])
        encrypted_message.extend(encrypted_block)

    # If there are leftovers, they are treated separately
    if plaintext_len % 64 != 0:
        j = plaintext_len // 64
        key_stream = my_chacha20_block(key, counter + j, nonce)
        block = bytearray(plaintext[j*64:], 'utf-8')
        encrypted_block = bytes([block[i] ^ key_stream[i] for i in range(len(block))])
        encrypted_message.extend(encrypted_block)

    print_serialized_block(encrypted_message)
    return encrypted_message


# Example of use
plaintext = "Ladies and Gentlemen of the class of '99: If I could offer you only one tip for the future, sunscreen would be it."
key = "00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f 10 11 12 13 14 15 16 17 18 19 1a 1b 1c 1d 1e 1f"
nonce = "00 00 00 00 00 00 00 4a 00 00 00 00"
encrypted_message = my_chacha20_encrypt(key, 1, nonce, plaintext)

000  6e 2e 35 9a 25 68 f9 80 41 ba 07 28 dd 0d 69 81   n.5.%h..A..(..i.
016  e9 7e 7a ec 1d 43 60 c2 0a 27 af cc fd 9f ae 0b   .~z..C`..'......
032  f9 1b 65 c5 52 47 33 ab 8f 59 3d ab cd 62 b3 57   ..e.RG3..Y=..b.W
048  16 39 d6 24 e6 51 52 ab 8f 53 0c 35 9f 08 61 d8   .9.$.QR..S.5..a.
064  07 ca 0d bf 50 0d 6a 61 56 a3 8e 08 8a 22 b6 5e   ....P.jaV....".^
080  52 bc 51 4d 16 cc f8 06 81 8c e9 1a b7 79 37 36   R.QM.........y76
096  5a f9 0b bf 74 a3 5b e6 b4 0b 8e ed f2 78 5e 42   Z...t.[......x^B
112  87 4d                                             .M


In [42]:
def my_chacha20_decrypt(key, counter, nonce, cyphertext):
    decrypted_message = bytearray()
    cyphertext_len = len(cyphertext)

    # Process 64 byte blocks
    for j in range(cyphertext_len // 64):
        key_stream = my_chacha20_block(key, counter + j, nonce)
        block = cyphertext[j*64:(j+1)*64]
        decrypted_block = bytes([block[i] ^ key_stream[i] for i in range(64)])
        decrypted_message.extend(decrypted_block)

    # If there are leftovers, they are treated separately
    if cyphertext_len % 64 != 0:
        j = cyphertext_len // 64
        key_stream = my_chacha20_block(key, counter + j, nonce)
        block = cyphertext[j*64:]
        decrypted_block = bytes([block[i] ^ key_stream[i] for i in range(len(block))])
        decrypted_message.extend(decrypted_block)

    print_serialized_block(decrypted_message)
    return decrypted_message

In [43]:
##TESTS ChaCha20 Block Functions
## https://www.rfc-editor.org/rfc/rfc8439.html#appendix-A

print("TEST1")
key = "00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00"
nonce = "00 00 00 00 00 00 00 00 00 00 00 00"
block_counter = 0
my_chacha20_block(key,block_counter,nonce, True)


print("TEST2")
key = "00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00"
nonce = "00 00 00 00 00 00 00 00 00 00 00 00"
block_counter = 1
my_chacha20_block(key,block_counter,nonce, True)

print("TEST3")
key = "00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 01"
nonce = "00 00 00 00 00 00 00 00 00 00 00 00"
block_counter = 1
my_chacha20_block(key,block_counter,nonce, True)

print("TEST4")
key = "00 ff 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00"
nonce = "00 00 00 00 00 00 00 00 00 00 00 00"
block_counter = 2
my_chacha20_block(key,block_counter,nonce, True)

print("TEST5")
key = "00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00"
nonce = "00 00 00 00 00 00 00 00 00 00 00 02"
block_counter = 0
my_chacha20_block(key,block_counter,nonce, True)


TEST1
add the original state
0x61707865	 0x3320646e	 0x79622d32	 0x6b206574	 
0x00000000	 0x00000000	 0x00000000	 0x00000000	 
0x00000000	 0x00000000	 0x00000000	 0x00000000	 
0x00000000	 0x00000000	 0x00000000	 0x00000000	 
after block operations
0xade0b876	 0x903df1a0	 0xe56a5d40	 0x28bd8653	 
0xb819d2bd	 0x1aed8da0	 0xccef36a8	 0xc70d778b	 
0x7c5941da	 0x8d485751	 0x3fe02477	 0x374ad8b8	 
0xf4b8436a	 0x1ca11815	 0x69b687c3	 0x8665eeb2	 
serialize
000  76 b8 e0 ad a0 f1 3d 90 40 5d 6a e5 53 86 bd 28   v.....=.@]j.S..(
016  bd d2 19 b8 a0 8d ed 1a a8 36 ef cc 8b 77 0d c7   .........6...w..
032  da 41 59 7c 51 57 48 8d 77 24 e0 3f b8 d8 4a 37   .AY|QWH.w$.?..J7
048  6a 43 b8 f4 15 18 a1 1c c3 87 b6 69 b2 ee 65 86   jC.........i..e.
TEST2
add the original state
0x61707865	 0x3320646e	 0x79622d32	 0x6b206574	 
0x00000000	 0x00000000	 0x00000000	 0x00000000	 
0x00000000	 0x00000000	 0x00000000	 0x00000000	 
0x00000001	 0x00000000	 0x00000000	 0x00000000	 
after block operations
0xbee7079f

bytearray(b'\xc2\xc6M7\x8c\xd567J\xe2\x04\xb9\xef\x93?\xcd\x1a\x8b"\x88\xb3\xdf\xa4\x96r\xabv[T\xee\'\xc7\x8a\x97\x0e\x0e\x95\\\x14\xf3\xa8\x8et\x1b\x97\xc2\x86\xf7_\x8f\xc2\x99\xe8\x14\x83b\xfa\x19\x8a9S\x1b\xedm')

In [44]:
##TESTS ChaCha20 Encryption / Decryption
## https://www.rfc-editor.org/rfc/rfc8439.html#appendix-A.2

print("Test Vector #2")
key = "00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 01"
nonce = "00 00 00 00 00 00 00 00 00 00 00 02"
block_counter = 1
plaintext = 'Any submission to the IETF intended by the Contributor for publication as all or part of an IETF Internet-Draft or RFC and any statement made within the context of an IETF activity is considered an "IETF Contribution". Such statements include oral statements in IETF sessions, as well as written and electronic communications made at any time or place, which are addressed to'

cyphertext = my_chacha20_encrypt(key, block_counter, nonce, plaintext)

print("CYPHER")
my_chacha20_decrypt(key, 1, nonce, cyphertext)


print("Test Vector #3")
key = "1c 92 40 a5 eb 55 d3 8a f3 33 88 86 04 f6 b5 f0 47 39 17 c1 40 2b 80 09 9d ca 5c bc 20 70 75 c0"
nonce = "00 00 00 00 00 00 00 00 00 00 00 02"
block_counter = 42
plaintext = r"'Twas brillig, and the slithy toves Did gyre and gimble in the wabe: All mimsy were the borogoves, And the mome raths outgrabe."
cyphertext = my_chacha20_encrypt(key, block_counter, nonce, plaintext)

print("CYPHER")
my_chacha20_decrypt(key, 42, nonce, cyphertext)
print()

Test Vector #2
000  a3 fb f0 7d f3 fa 2f de 4f 37 6c a2 3e 82 73 70   ...}../.O7l.>.sp
016  41 60 5d 9f 4f 4f 57 bd 8c ff 2c 1d 4b 79 55 ec   A`].OOW...,.KyU.
032  2a 97 94 8b d3 72 29 15 c8 f3 d3 37 f7 d3 70 05   *....r)....7..p.
048  0e 9e 96 d6 47 b7 c3 9f 56 e0 31 ca 5e b6 25 0d   ....G...V.1.^.%.
064  40 42 e0 27 85 ec ec fa 4b 4b b5 e8 ea d0 44 0e   @B.'....KK....D.
080  20 b6 e8 db 09 d8 81 a7 c6 13 2f 42 0e 52 79 50    ........./B.RyP
096  42 bd fa 77 73 d8 a9 05 14 47 b3 29 1c e1 41 1c   B..ws....G.)..A.
112  68 04 65 55 2a a6 c4 05 b7 76 4d 5e 87 be a8 5a   h.eU*....vM^...Z
128  d0 0f 84 49 ed 8f 72 d0 d6 62 ab 05 26 91 ca 66   ...I..r..b..&..f
144  42 4b c8 6d 2d f8 0e a4 1f 43 ab f9 37 d3 25 9d   BK.m-....C..7.%.
160  c4 b2 d0 df b4 8a 6c 91 39 dd d7 f7 69 66 e9 28   ......l.9...if.(
176  e6 35 55 3b a7 6c 5c 87 9d 7b 35 d4 9e b2 e6 2b   .5U;.l\..{5....+
192  08 71 cd ac 63 89 39 e2 5e 8a 1e 0e f9 d5 28 0f   .q..c.9.^.....(.
208  a8 ca 32 8b 35 1c 3c 76 59 89 cb cf 3d aa 8b